In [ ]:
from bs4 import BeautifulSoup
import urllib.request
from urllib.request import urlopen
import time
import pandas as pd
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Collect article lists

In [ ]:
# 한겨레 2024년 1월 1일부터 12월 31일 사이 '사다리'라는 키워드로 검색한 기사 수집하기
# 이 작업을 수행하기 전에 링크 전체 주소에 페이지 표시가 포함되어있는지 확인 필요

FIRST_PAGE = 'https://search.hani.co.kr/search/newslist?searchword=%EC%82%AC%EB%8B%A4%EB%A6%AC&startdate=20240101&enddate=20241231&page=1&sort=desc'

# TARGET_URL 에서는 페이지 번호가 1부터 반복해서 입력될 수 있도록 제거(페이지 번호 위치가 어디인지 위, 아래 비교 필요)
TARGET_URL_KEYWORD = 'https://search.hani.co.kr/search/newslist?searchword=%EC%82%AC%EB%8B%A4%EB%A6%AC&startdate=20240101&enddate=20241231&page=' # 메인 + 키워드
TARGET_URL_REST = '&sort=desc' # 페이지 및 기사 정렬
TARGET_URL = TARGET_URL_KEYWORD + TARGET_URL_REST

# 한겨레의 경우 다른 곳과는 달리 페이지번호가 빠진 링크는 오류가 뜨므로-기사 수집에는 문제없음-위 FIRST_PAGE 링크로 기사 개수 등을 확인할 것
print(TARGET_URL)

https://search.hani.co.kr/search/newslist?searchword=%EC%82%AC%EB%8B%A4%EB%A6%AC&startdate=20240101&enddate=20241231&page=&sort=desc


In [ ]:
# 기사 제목, URL, 게시 날짜를 저장할 리스트 초기화
TITLE_OF_ARTICLE = []
URL_OF_ARTICLE = []
DATE_OF_ARTICLE = []

# 첫 페이지의 URL과 사용자 에이전트를 설정하여 요청 객체 생성
url = urllib.request.Request(FIRST_PAGE, # URL에 대한 요청 생성
                             headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'})
html = urllib.request.urlopen(url)      # 생성된 요청 객체를 사용하여 웹 페이지 열기
soup = BeautifulSoup(html, 'html.parser', from_encoding='utf-8')     # BeautifulSoup 객체를 사용하여 HTML 파싱, 인코딩 설정 포함

for element in soup.select('strong'):   # <strong> 태그를 포함하는 모든 요소 선택하기: 기사 제목 추출
    text = element.get_text(strip=True)  # 각 요소에서 텍스트 추출 및 공백 제거
    TITLE_OF_ARTICLE.append(text)  # 추출된 텍스트를 제목 리스트에 추가

for element in soup.select('a.flex-inner'):     # class='flex-inner'를 포함하는 모든 <a> 태그 선택하기: 기사 URL 추출
    href = element.get('href')   # href 속성 추출, 없으면 'Not found' 처리
    URL_OF_ARTICLE.append(href)      # 추출된 href를 URL 리스트에 추가

for element in soup.select('span.article-date'):    # class='article-date'를 포함하는 모든 <span> 태그 선택하기: 게시 날짜 추출
    date = element.get_text(strip=True).split()[0]  # 텍스트 추출 후 공백 기준 분리, 첫 번째 요소 선택
    DATE_OF_ARTICLE.append(date)  # 추출된 날짜를 날짜 리스트에 추가

# 수집된 기사 제목, URL, 게시 날짜 리스트 출력
print(TITLE_OF_ARTICLE)
print(URL_OF_ARTICLE)
print(DATE_OF_ARTICLE)

['대국민 ‘관저 농성’과 경찰의 결기 [전국 프리즘]', '정부가 ‘빌라’ 좀 사라는데 사도 될까요? [집문집답]', '‘오징어 게임2’ 시즌1과 비슷한 듯 다르다…관전 포인트는?', '불법 계엄군의 가족의 조건을 생각한다 [.txt]', '끊어진 계층 이동사다리…10명 중 3명은 ‘저소득층 수렁’', '뒤집힌 계엄, 현장엔 그들이 있었다 [시민편집인의 눈]', '기고∥가상자산 과세, 득보다 실이 클 수 있다', '한동훈표 ‘이준석 시즌2’…2030 남성을 잡아라', '인천·김포 건설현장서 잇단 사망…크레인 해체하다 외벽 도색하다 추락', '윤 대통령 “한국이 ‘녹색사다리’ 역할…그린 ODA 확대”']
['https://www.hani.co.kr/arti/opinion/column/1175745.html', 'https://www.hani.co.kr/arti/economy/property/1175679.html', 'https://www.hani.co.kr/arti/culture/culture_general/1174994.html', 'https://www.hani.co.kr/arti/culture/book/1174172.html', 'https://www.hani.co.kr/arti/economy/economy_general/1173833.html', 'https://www.hani.co.kr/arti/opinion/column/1172071.html', 'https://www.hani.co.kr/arti/opinion/readercolumn/1169957.html', 'https://www.hani.co.kr/arti/politics/polibar/1169404.html', 'https://www.hani.co.kr/arti/area/capital/1168706.html', 'https://www.hani.co.kr/arti/politics/politics_general/1168220.html']
['2024-12-31', '2024-12-31', '20

In [ ]:
# Make a new function for automatic scrapping

def get_link_from_news_title(page_num, URL):
    # 지정된 페이지 수만큼 반복
    for i in range(page_num):
        current_page_num = 1 + i # 현재 페이지 번호 계산
        position = URL.index('page=') # URL에서 'page='의 위치를 찾아 페이지 번호를 업데이트한 새 URL 생성
        URL_with_page_num = URL[:position+5] + str(current_page_num) + URL[position+5:]
        url = urllib.request.Request(URL_with_page_num,
                                     headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'})
        html = urllib.request.urlopen(url)
        soup = BeautifulSoup(html, 'html.parser', from_encoding='utf-8') # BeautifulSoup 객체를 사용하여 HTML 파싱

        strong_elements = soup.find_all('strong')
        for element in strong_elements:
            text = element.get_text().strip()
            TITLE_OF_ARTICLE.append(text)

        a_class_elements = soup.find_all('a', class_='flex-inner')
        for element in a_class_elements:
            href = element['href'] if element else 'Not found'
            URL_OF_ARTICLE.append(href)

        span_elements = soup.find_all('span', class_='article-date')
        for element in span_elements:
            date = element.get_text().split()[0]
            DATE_OF_ARTICLE.append(date)

        # 서버 부하를 줄이기 위해 일정 페이지마다 일시 정지
        if i % 10 == 0:
            time.sleep(2)
        # 진행 상태 로깅
        if i % 10 == 0:
            print(f'{i*10}nd article complete')

In [ ]:
# 전체 기사 수를 받아 필요한 페이지 수를 계산하는 함수 정의
# 아래 코드는 내가 보고있는 페이지의 게시물 개수에 따라 숫자가 달라지므로 반드시 확인

def get_total_num_of_article(number):
    if(number % 10 >= 1):   # 10으로 나눈 나머지가 1 이상이면 (즉, 나누어떨어지지 않으면)
        page_num = int((number / 10) + 1)   # 몫에 1을 더하여 추가 페이지 확보 (반올림 효과)
    else:   # 10으로 나누어떨어지는 경우
        page_num = int(number / 10)     # 정확히 나눈 몫이 곧 페이지 수
    return page_num

page_num = get_total_num_of_article(134)
page_num

14

In [ ]:
# 기사 제목, URL, 게시 날짜를 저장할 리스트를 초기화
TITLE_OF_ARTICLE = []
URL_OF_ARTICLE = []
DATE_OF_ARTICLE = []

# get_link_from_news_title 함수를 호출하여 지정된 페이지 수와 URL을 기반으로 뉴스 기사의 제목, URL, 게시 날짜를 수집
get_link_from_news_title(page_num, TARGET_URL)

0nd article complete
100nd article complete


In [ ]:
df = pd.DataFrame(list(zip(DATE_OF_ARTICLE, URL_OF_ARTICLE, TITLE_OF_ARTICLE)), columns =['Date', 'Url', 'Title'])  # 수집한 리스트를 각각 (날짜, URL, 제목) 튜플로 묶은 후 데이터프레임 처리

print(len(df))  # 생성된 DataFrame의 행 개수 출력 (=기사 수) / 단 한겨레는 현재 검색결과보다 검색 결과의 기사 개수가 1개 덜 나오는 문제가 있기 설정한 기간에 나오는 기사 개수 +1 이 정확한 개수임
df  # 전체 DataFrame 출력

134


,Date,Url,Title
0,2024-12-31,https://www.hani.co.kr/arti/opinion/column/117...,대국민 ‘관저 농성’과 경찰의 결기 [전국 프리즘]
1,2024-12-31,https://www.hani.co.kr/arti/economy/property/1...,정부가 ‘빌라’ 좀 사라는데 사도 될까요? [집문집답]
2,2024-12-26,https://www.hani.co.kr/arti/culture/culture_ge...,‘오징어 게임2’ 시즌1과 비슷한 듯 다르다…관전 포인트는?
3,2024-12-20,https://www.hani.co.kr/arti/culture/book/11741...,불법 계엄군의 가족의 조건을 생각한다 [.txt]
4,2024-12-18,https://www.hani.co.kr/arti/economy/economy_ge...,끊어진 계층 이동 사다리…10명 중 3명은 ‘저소득층 수렁’
...,...,...,...
129,2024-01-06,https://www.hani.co.kr/arti/specialsection/esc...,누가 대전을 ‘노잼 도시’라 했나…소소한 재미에 멈춰서는 곳 [ESC]
130,2024-01-03,https://www.hani.co.kr/arti/area/capital/11227...,"아파트 관리소장, 안전모 미착용 숨기려 현장 조작해 기소"
131,2024-01-02,https://www.hani.co.kr/arti/politics/politics_...,윤 대통령 “금투세 폐지 추진”…여야 합의 뒤집고 포퓰리즘 질주
132,2024-01-02,https://www.hani.co.kr/arti/opinion/column/112...,"[슬픈 경쟁, 아픈 교실] 학원 가는 길"


In [ ]:
# DataFrame을 CSV 파일로 저장
df.to_csv("/content/drive/MyDrive/CLASS/202502_NLP/Hani.csv",     # 저장할 파일 경로 (Google Drive 내 지정 경로)
    index=False,    # 행 번호(인덱스)를 저장하지 않음
    encoding='utf-8-sig'    # 인코딩 방식: 한글 깨짐 방지를 위해 'utf-8-sig' 사용
)

## Collect texts from the article

In [ ]:
url = urllib.request.Request(df['Url'][0],
                             headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.85 Safari/537.36'})

source_code_from_url = urllib.request.urlopen(url)  # 설정한 요청으로부터 소스 코드를 가져옴
soup = BeautifulSoup(source_code_from_url, 'html.parser', from_encoding='utf-8')    # 가져온 소스 코드를 BeautifulSoup을 사용하여 파싱, 이때 HTML의 구조를 분석하기 위해 'html.parser' 사용
content = soup.select('div.article-text > p') # 기사 내용이 담긴 div 태그의 클래스 'article_txt'를 선택

# 선택된 단락들로부터 텍스트를 수집하여 저장할 리스트 초기화
CONTENT = []
for item in content:
    CONTENT.append(item.get_text()) # 각 단락의 텍스트를 CONTENT 리스트에 추가

CONTENT = " ".join(CONTENT)     # 리스트에 저장된 단락 텍스트들을 하나의 문자열로 결합하되 " ".join 을 활용하여 단락 사이의 간격을 공백 한 칸으로 변경
CONTENT

'김기성 | 수도권데스크 \xa02009년 1월19일 서울 용산4구역 재개발 지역 철거민들은 상가 건물 ‘남일당’ 옥상에 망루를 짓고 농성을 시작했다. 벼랑 끝까지 내몰린 이들의 마지막 선택이자, 살아갈 길만은 열어달라는 처절한 몸부림이었다. 그러나 국가는 자본 편에 섰다. 이튿날 새벽 3시30분 ‘대테러 담당’인 경찰특공대가 도착했다. 특공대는 이들을 국민으로 보지 않았다. 지게차에 실린 컨테이너가 망루로 올려졌고, 무자비한 진압이 자행됐다. 인화물질이 가득했던 망루에선 불이 났고, 철거민 5명과 경찰특공대 1명이 숨졌다. 야만적 진압이 부른 ‘참사’였다. 그로부터 7개월 뒤인 2009년 8월4~5일. 나는 다시 열린 ‘야만의 문’을 현장에서 들여다봤다. 600여명의 노동자가 ‘해고는 살인’이라고 외치며 농성을 벌이던 경기도 평택 쌍용자동차 공장 하늘 위로 헬기가 떠올랐다. 30m 높이로 저공비행 하며 최루액을 무차별 살포했다. 공장 밖에서는 1000여명의 경찰이 노동자들을 포위했다. 특공대는 세 방향에서 사다리차를 놓아 공장으로 진입했다. 대테러 장비인 테이저건과 다목적발사기 등으로 무장한 특공대는 전쟁터에서 적을 섬멸하듯 노동자들을 공격했다. 뜨거운 여름 햇살에 달궈진 공장 철판 지붕 위로 내몰린 노동자들은 곤봉에 얻어맞고, 군홧발에 짓이겨졌다. 살인적 진압으로 농성장은 초토화됐다. 노동자들은 공장 밖으로 끌려 나와 줄줄이 연행됐다. 살인적 진압이었다. 이후 해고된 노동자들이 하나둘 극단적 선택을 했다. 그 가족들을 포함해 30명이 넘는 이들이 세상을 등졌다. 이 역시 ‘참사’로 기록할 수밖에 없었다. 이처럼 경찰의 공권력은 막강했다. 아무리 가엾은 처지에 내몰려 울부짖어도, 자본과 권력에 저항하는 시민과 노동자들에겐 절대 자비를 베풀지 않아 왔다. 다수의 국민을 불편하게 하고, 국가의 근간을 흔들 수 있다는 판단을 내리면 최정예 대테러 부대까지 동원해 가차 없이 농성을 진압해왔다. 그럼 근현대사에서 찾아보기 아주 힘든 ‘희대의 농성’을 벌이는 ‘내란 수

In [ ]:
# 기사 URL 목록을 입력으로 받아 각 URL에서 텍스트를 추출하는 함수

def get_texts_from_news(URL):
    # URL 목록의 각 URL에 대하여 반복 실행
    for i in range(len(URL)):
        # 현재 URL에 대한 요청 객체 생성
        url = urllib.request.Request(URL[i], headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.85 Safari/537.36'})
        try:
            source_code_from_url = urllib.request.urlopen(url)  # 요청을 보내고 응답으로부터 HTML 소스 코드를 가져옴
            soup = BeautifulSoup(source_code_from_url, 'html.parser', from_encoding='utf-8')    # BeautifulSoup을 사용하여 HTML 소스 코드 파싱
            content = soup.select('div.article-text > p') # 기사 내용이 담긴 div 태그의 클래스 'article_txt'를 선택

            CONTENT = []    # 기사 내용을 저장할 리스트 초기화
            for item in content:
                    CONTENT.append(item.get_text())     # 각 p 태그의 텍스트를 리스트에 추가
            CONTENT = " ".join(CONTENT)     # 리스트에 저장된 텍스트를 하나의 문자열로 합침
            CONTENT_OF_ARTICLE.append(CONTENT)  # 최종 추출된 기사 본문을 리스트에 추가

        except Exception as e: # 예외 처리
            print(f"article No. {i} Error occurred with URL {URL}: {e}") # 오류가 나는 기사 인덱스 출력
            CONTENT_OF_ARTICLE.append('NONE') # 오류 발생 시 'NONE' 저장

        # 서버 부하 방지 및 진행 모니터링
        if i % 10 == 0:
            time.sleep(1)
        if i % 100 == 0:
            print(f'{i} articles processed')

In [ ]:
# Input the list of article links

CONTENT_OF_ARTICLE = []

get_texts_from_news(URL_OF_ARTICLE)

0 articles processed
100 articles processed


In [ ]:
df = pd.DataFrame(list(zip(DATE_OF_ARTICLE, URL_OF_ARTICLE, TITLE_OF_ARTICLE, CONTENT_OF_ARTICLE)), columns =['Date', 'Url', 'Title', 'Content'])

print(len(df))
df.head()

134


,Date,Url,Title,Content
0,2024-12-31,https://www.hani.co.kr/arti/opinion/column/117...,대국민 ‘관저 농성’과 경찰의 결기 [전국 프리즘],김기성 | 수도권데스크 2009년 1월19일 서울 용산4구역 재개발 지역 철거민들...
1,2024-12-31,https://www.hani.co.kr/arti/economy/property/1...,정부가 ‘빌라’ 좀 사라는데 사도 될까요? [집문집답],"안녕하세요, 한겨레 집문집답 운영자인 소심한 무주택자 김소심입니다. 25일 한국부동..."
2,2024-12-26,https://www.hani.co.kr/arti/culture/culture_ge...,‘오징어 게임2’ 시즌1과 비슷한 듯 다르다…관전 포인트는?,“익숙하면서도 새로운 것들을 만들어내려고 했습니다.” 올해 최대 기대작인 넷플릭스 ...
3,2024-12-20,https://www.hani.co.kr/arti/culture/book/11741...,불법 계엄군의 가족의 조건을 생각한다 [.txt],12·3 내란사태의 전개를 보면서 한편 궁금해지는 건 군인 가족의 근황이다. 행위자...
4,2024-12-18,https://www.hani.co.kr/arti/economy/economy_ge...,끊어진 계층 이동 사다리…10명 중 3명은 ‘저소득층 수렁’,우리 국민 10명 중 3명이 6년 동안 소득 하위 20%를 벗어나지 못하고 가난한 ...


In [ ]:
df['Content'].replace('NONE', np.nan, inplace=True) # 'Content' 열에서 'NONE' 문자열을 NaN 값으로 대체
df['Content'].replace('', np.nan, inplace=True) # 'Content' 열에서 빈 문자열을 NaN 값으로 대체
df.dropna(subset=['Content'], inplace=True) # 'Content' 열에 NaN 값을 포함하는 모든 행을 삭제

# 처리 후 남은 행의 개수 출력
print(len(df))

127


/tmp/ipython-input-1632476055.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Content'].replace('NONE', np.nan, inplace=True) # 'Content' 열에서 'NONE' 문자열을 NaN 값으로 대체
/tmp/ipython-input-1632476055.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df

In [ ]:
df = df[~df["Content"].str.contains(r"사다리차",     # "사다리차"라는 단어가 "Content" 컬럼에 포함된 행을 필터링
                                    case=False,     # 대소문자 구분 없이 검색
                                    na=False)]      # 결측값(NaN)은 False로 간주하여 포함되지 않도록 설정

print(len(df))
df.to_csv("/content/drive/MyDrive/CLASS/202502_NLP/Hani_final.csv",index=False, encoding='utf-8-sig')

114
